In [1]:

import OpenDartReader
import requests as rq
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import datetime,timedelta
import os
from datetime import date
from dateutil.relativedelta import relativedelta
from html_table_parser import parser_functions as parser
import xlwings as xw
from tqdm import tqdm
import FinanceDataReader as fdr
import openpyxl

api_key = '08d5ae18b24d9a11b7fd67fb0d79c607f1c88464'
dart = OpenDartReader(api_key)

# fr = (date.today() + relativedelta(days =-5)).strftime("%Y%m%d") #시작일 date.today 는 오늘 날짜가 나옴, 5년전에 날짜로 지정
yester = (datetime.today()-timedelta(1)).strftime("%Y%m%d") #오늘
print(yester)
to = datetime.today().strftime("%Y%m%d") #오늘
df = dart.list(start=yester, end=to)
df = df[df['report_nm']== '임원ㆍ주요주주특정증권등소유상황보고서']
table1 = pd.DataFrame()

a = 0
for i in tqdm(df['rcept_no']):
    try:
        recept = df['rcept_no'].iloc[a]
        # recept = '20240709000447'
        res = dart.sub_docs(recept)['url'][3]
        
        call = rq.get(res)
        soup = BeautifulSoup(call.text, "html.parser")
        before = soup.select('body > table')
        if '사유' in parser.make2d(before[-1])[0][0]:
            table = parser.make2d(before[-1])
        elif '사유' in parser.make2d(before[-2])[0][0]:
            table = parser.make2d(before[-2])
        data = pd.DataFrame(table,columns=table[1])
        data = data.drop(index=[0,1])
        data = data.drop(data.index[-1:])
        data.insert(0,'기업명',df['corp_name'].iloc[a])
        data.insert(9,'제출인',df['flr_nm'].iloc[a])
        a += 1
        table1 = pd.concat([table1,data])
        time.sleep(1)
    except Exception as e:
        print(e)
table1 = table1[table1['보고사유'].str.contains('장내매수|장내매도')]
table1 = table1[~table1['기업명'].str.contains('리츠')]
table1['변동일*'] = table1['변동일*'].str.strip().str.replace('년 ','.').str.replace('월 ','.').str.replace('일','')
table1['취득/처분단가(원)**'] = table1['취득/처분단가(원)**'].str.strip().str.replace('원','').str.replace('(','').str.replace(')','').str.replace('취득','').str.replace('/','').str.replace(',','').str.replace('해당없음','')
# table1['취득/처분단가(원)**'] = table1['취득/처분단가(원)**'].astype(float)
# table1['증감'] = table1['증감'].str.replace(',','').astype(int)
# table1['금액'] = round((table1['증감'] * table1['취득/처분단가(원)**'])/100000000,2)
table1.insert(0,'조회날짜',to)
file_name = f'C:\투자기록\정리\코딩\장내매수\장내매수_종합.xlsx'
writer = pd.ExcelWriter(file_name, mode='a', engine='openpyxl', if_sheet_exists='overlay')
table1.to_excel(
    writer, 
    sheet_name='Sheet1',
    startcol = 0,
    startrow = writer.sheets['Sheet1'].max_row,
    index=False, 
    na_rep = '',      # 결측값을 ''으로 채우기
    inf_rep = '',     # 무한값을 ''으로 채우기
    header = None

    )


writer.close()
    

20241008


100%|██████████| 67/67 [01:56<00:00,  1.74s/it]
C:\Users\codek\AppData\Local\Temp\ipykernel_5984\4075465710.py:54: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  table1['취득/처분단가(원)**'] = table1['취득/처분단가(원)**'].str.strip().str.replace('원','').str.replace('(','').str.replace(')','').str.replace('취득','').str.replace('/','').str.replace(',','').str.replace('해당없음','')


In [9]:

from pykrx import stock
tickers = stock.get_index_ticker_list(market='KOSDAQ')
tickers

JSONDecodeError: Expecting value: line 15 column 3 (char 33)

In [4]:
file_name = f'C:\파이썬\주식관련\종목정보\기업재무실적\장내매수.xlsx'
writer = pd.ExcelWriter(file_name, mode='a', engine='openpyxl', if_sheet_exists='overlay')
table1.to_excel(
    writer, 
    sheet_name='Sheet1',
    startcol = 0,
    startrow = writer.sheets['Sheet1'].max_row,
    index=False, 
    na_rep = '',      # 결측값을 ''으로 채우기
    inf_rep = '',     # 무한값을 ''으로 채우기
    header = None

    )

writer.close()

In [2]:
import requests
import pandas as pd

class KrxHistoricalDailyPriceDataDownloader:

    def __init__(self):
        self._headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36',
        }

        self._stocks = None
        self._stocks_delisted = None

        self._bld = 'dbms/MDC/STAT/standard/MDCSTAT01701'

    def get_stocks(self):
        data = {
            'mktsel': 'ALL',
            'typeNo': '0',
            'searchText': '',
            'bld': 'dbms/comm/finder/finder_stkisu',
        }
        url = 'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'
        response = requests.post(url, data, headers=self._headers)
        df = pd.json_normalize(response.json()['block1'])
        df = df.set_index('short_code')
        return df

    def get_stocks_delisted(self):
        data = {
            'mktsel': 'ALL',
            'searchText': '',
            'bld': 'dbms/comm/finder/finder_listdelisu',
        }
        url = 'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'
        response = requests.post(url, data, headers=self._headers)
        df = pd.json_normalize(response.json()['block1'])
        df = df.set_index('short_code')
        return df

    @property
    def stocks(self):
        if self._stocks is None:
            self._stocks = self.get_stocks()
        return self._stocks

    @property
    def stocks_delisted(self):
        if self._stocks_delisted is None:
            self._stocks_delisted = self.get_stocks_delisted()
        return self._stocks_delisted

    def get_full_code(self, symbol):
        if symbol in self.stocks.index:
            return self.stocks.loc[symbol]['full_code']
        if symbol in self.stocks_delisted.index:
            return self.stocks_delisted.loc[symbol]['full_code']
        raise ValueError('No full_code found for given symbol %s' % symbol)

    def download(self, symbol, start_date=None, end_date=None):
        if start_date is None:
            start_date = pd.Timestamp(1980, 1, 1)
        if end_date is None:
            end_date = pd.Timestamp.now().normalize() + pd.Timedelta(1, unit='day')

        full_code = self.get_full_code(symbol)

        url = 'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'
        data = {
            'bld': self._bld,
            'isuCd': full_code,
            'isuCd2': '',
            'strtDd': start_date.strftime("%Y%m%d"),
            'endDd': end_date.strftime("%Y%m%d"),
            'share': '1',
            'money': '1',
            'csvxls_isNo': 'false',
        }
        response = requests.post(url, data, headers=self._headers)
        df = pd.json_normalize(response.json()['output'])

        if df.shape[0] == 0:
            return None

        column_names = {
            'TRD_DD':'Date',
            'ISU_CD':'Code',
            'ISU_NM':'Name',
            'MKT_NM':'Market',
            'SECUGRP_NM':'SecuGroup',
            'TDD_CLSPRC':'Close',
            'FLUC_TP_CD':'UpDown',
            'CMPPRVDD_PRC':'Change',
            'FLUC_RT':'ChangeRate',
            'TDD_OPNPRC':'Open',
            'TDD_HGPRC':'High',
            'TDD_LWPRC':'Low',
            'ACC_TRDVOL':'Volume',
            'ACC_TRDVAL':'Amount',
            'MKTCAP':'MarCap',
            'CMPPREVDD_PRC': 'Change',
            'LIST_SHRS': 'Shares',
        }
        df = df.rename(columns=column_names)

        df['Date'] = pd.to_datetime(df['Date'])
        df['ChangeRate'] = pd.to_numeric(df['ChangeRate'].str.replace(',', ''))

        int_columns = [
            'Close',
            'UpDown',
            'Change',
            'Open',
            'High',
            'Low',
            'Volume',
            'Amount',
            'MarCap',
            'Shares'
        ]

        for col in int_columns:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col].str.replace(',', ''), errors='coerce')
                
        df.set_index('Date', inplace=True)

        return df

In [3]:
krx_downloader = KrxHistoricalDailyPriceDataDownloader()

In [3]:
import FinanceDataReader as fdr

krx = fdr.StockListing('KRX')
krx

JSONDecodeError: Expecting value: line 16 column 3 (char 36)